# 12. Split Dashcam Footage

Read a directory full of MP4 video files from a dashcam, split them into frames at the required intervals (from the 60fps footage) while correlating them to coordinate metadata that recorded in the accompanying NMEA file.

## Configuration

Any configuration that is required to run this notebook can be customized in the next cell

In [1]:
# Name of a folder containing input MP4 videos and their corresponding NMEA files,
# imported from the dash camera.  This notebook will assume this folder is found
# inside the 'data_sources' folder.
# We will create a "split" subdirectory inside the import_directory, containing
# each of the images, plus a "metadata.csv" file to describe each of them in terms
# of latitude/longitude, altitude, and heading
import_directory = 'dashcam_tour_mount_eliza'
#import_directory = 'dashcam_tour_frankston'

# Required frames per second for output images, reduced down from 60 fps
output_fps = 5

## Explanation of NMEA data

Sample NMEA data, repeats every 60 seconds:
``` NMEA sample
$GPGGA,004008.00,3810.18002,S,14506.69684,E,1,11,0.85,43.6,M,-2.7,M,,*6B
$GPGSA,A,3,30,14,07,13,17,194,193,195,19,21,15,,1.47,0.85,1.20*3E
$GPGSV,5,1,20,01,10,085,21,05,09,275,15,07,35,078,34,08,10,142,14*7B
$GPGSV,5,2,20,09,04,021,08,13,47,236,23,14,75,226,32,15,19,225,21*77
$GPGSV,5,3,20,17,43,008,24,19,22,354,29,20,01,305,,21,13,104,24*74
$GPGSV,5,4,20,28,,,32,30,69,110,38,41,13,288,,50,42,332,*42
$GPGSV,5,5,20,53,22,297,,193,29,356,26,194,74,274,32,195,19,355,31*4E
$GSENSORD,0.089,0.123,-0.140
$GPRMC,004009.00,A,3810.17224,S,14506.70055,E,29.706,20.61,240921,,,A*74
```

http://aprs.gids.nl/nmea/

* GPGGA - Global Positioning System Fix Data

    - Time "004001.00" => The time is 00:40:01 in GMT
    - **Latitude "3810.18002,S"**
    - **Longitude "14506.69684,E"**
    - Fix Quality "1" = GPS fix.  0=Invalid, 2=DGPS fix.
    - Number of Satellites "11" = 11 Satellites are in view
    - Horizontal Dilution of Precision (HDOP) "0.85"
    - **Altitude "43.6,M"**
    - Height of geoid above WGS84 ellipsoid "-2.7,M"
    - DGPS reference station id ""
    - Checksum "*6B"
    
* GPRMC - Recommended minimum specific GPS/Transit data
    - Time "004009.00" => The time is 00:40:09 in GMT
    - Validity "A" => A=OK, V=invalid
    - **Latitude "3810.17224,S"**
    - **Longitude "14506.70055,E"**
    - Speed in knots "29.706" ==> converts to 55kmph. 1 knot = 1.852 kmph.
    - **True course "20.61" => bearing**
    - Date stamp "240921" => 24th of September 2021
    - Variation ""
    - East/West ""
    - Checksum "A*74"
    
* GPGSA - GPS DOP and active satellites
    * Ignore
    
* GPGSV - GPS Satellites in view
    * Ignore
    
* GSENSORD - Sensor data, possibly specific to the device
    * Ignore  
    
We are interested in latitude and longitude, of course, which can come from either GPGGA or GPRMC.  We are also interested in altitude (from GPGGA) and true course (from GPRMC) in case they come in handy.

There are 61 instances of GPGGA in the file, and 60 instances of GPRMC in between.  They agree on the co-ords as at
each timestamp.  We might as well parse them into a dictionary using the timestamp as the key.

## Code

In [2]:
# General imports
import os
import sys

module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)
    
# Import local modules
import osm_gsv_utils.dashcam_parser as dashcam_parser
import osm_gsv_utils.lane_detection as lane_detection

In [3]:
# Derived paths

# Full path to the directory containing the MP4 videos and NMEA files
dashcam_dir = os.path.join(module_path_root, 'data_sources', import_directory)

# "Split" subdirectory where the output frames will be created, along with a "metadata.csv"
# with metadata about each frame, loaded and interpolated from the NMEA files
output_dir  = os.path.join(dashcam_dir, 'split')

In [4]:
# Initialise an object to parse dashcam footage and correlate it with NMEA data
parser = dashcam_parser(source_fps=60, write_lane_images=True)

In [5]:
# Split all videos in the directory at the required output frames per second
parser.split_videos(dashcam_dir, output_dir, output_fps=output_fps, suffix='MP4', verbose=False)

# First progress bar shows progress through the input video files
# Subsequent progress bar shows progress within an individual video

# Ignore occasional warning "processing line" due to missing fields in NMEA file.
# As long as there are only a few of these, the values will be interpolated
# from nearby entries.

  0%|          | 0/45 [00:00<?, ?it/s]

FILE210924-100801F:   0%|          | 0/3612 [00:00<?, ?it/s]

FILE210924-100902F:   0%|          | 0/3612 [00:00<?, ?it/s]

FILE210924-101002F:   0%|          | 0/3612 [00:00<?, ?it/s]

FILE210924-101102F:   0%|          | 0/3612 [00:00<?, ?it/s]

...

FILE210924-104811F:   0%|          | 0/3612 [00:00<?, ?it/s]

FILE210924-104911F:   0%|          | 0/3612 [00:00<?, ?it/s]

FILE210924-105012F:   0%|          | 0/3612 [00:00<?, ?it/s]

FILE210924-105112F:   0%|          | 0/3612 [00:00<?, ?it/s]

FILE210924-105212F:   0%|          | 0/3612 [00:00<?, ?it/s]